<a href="https://colab.research.google.com/github/utk468/micro_organism_deeplearning/blob/main/micro_organism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
import shutil
from shutil import copytree, rmtree
from pathlib import Path

In [5]:
def prepare_data(source_dir, target_dir, split_ratio=0.8):

    if os.path.exists(target_dir):
        rmtree(target_dir)
    os.makedirs(target_dir)

    classes = os.listdir(source_dir)
    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        if os.path.isdir(cls_path):
            images = os.listdir(cls_path)
            train_size = int(len(images) * split_ratio)

            os.makedirs(os.path.join(target_dir, 'train', cls))
            os.makedirs(os.path.join(target_dir, 'validation', cls))

            for i, img in enumerate(images):
                src_path = os.path.join(cls_path, img)
                if os.path.isfile(src_path):
                    if i < train_size:
                        dest_path = os.path.join(target_dir, 'train', cls, img)
                    else:
                        dest_path = os.path.join(target_dir, 'validation', cls, img)
                    shutil.copy(src_path, dest_path)

source_dir = "/content/drive/MyDrive/Colab Notebooks/Micro_Organism"
target_dir = "organized_dataset"
prepare_data(source_dir, target_dir)

In [6]:

train_dir = os.path.join(target_dir, 'train')
validation_dir = os.path.join(target_dir, 'validation')

In [10]:


train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode="categorical"
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, target_size=(224, 224), batch_size=32, class_mode="categorical"
)



Found 636 images belonging to 8 classes.
Found 163 images belonging to 8 classes.


In [13]:

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False


model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(len(train_generator.class_indices), activation="softmax"),
])


model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)


early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)


history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping],
)


accuracy = model.evaluate(validation_generator)[1] * 100
print(f"Validation Accuracy: {accuracy:.2f}%")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.1395 - loss: 2.7193 - val_accuracy: 0.1963 - val_loss: 2.0216
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.2047 - loss: 2.2832 - val_accuracy: 0.2699 - val_loss: 1.8623
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.2362 - loss: 2.1221 - val_accuracy: 0.3558 - val_loss: 1.7639
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.3075 - loss: 1.8869 - val_accuracy: 0.3865 - val_loss: 1.6818
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.3405 - loss: 1.8093 - val_accuracy: 0.4110 - val_loss: 1.6131
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.3848 - loss: 1.7231 - val_accuracy: 0.4479 - val_loss: 1.5531
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.3803 - loss: 1.6888 - val_accuracy: 0.4908 - val_loss: 1.5036
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.4334 